In [1]:
import os
from pathlib import Path
import pandas as pd
from obspy.core import read, UTCDateTime, Stream
import obspy
from preprocessing_tools import *

In [2]:
directorio_recortado = "Datos_Trim/"
evid = "2018.08.12.14.42.09" # Sismo mas fuerte
evdir = directorio_recortado + evid
catalog_sub = 'catalog.dat'
catalogSSN = 'catalogSSN.dat'

# Generados durante el preprocesamiento
station_file = "station.csv"
ev_st = "ev_st.csv"

sacdir = "%s/RT"%evdir
outdir = evdir

P = Path(sacdir)
df_catalog = read_catalog(catalogSSN)
stations_meta = pd.read_csv(station_file)
ev_st_df = pd.read_csv(ev_st)

st = obspy.read(sacdir+'/*')

In [3]:
ev_row = df_catalog.loc[df_catalog['date_time_id'] == evid]

In [4]:
ev_row

,date_time,lat,lon,depth,mag,corr,date_time_id
0,2018-08-12T14:42:09.000000Z,17.11,-100.84,44.3,5.2,"[0.9316, 51.61, 20180812144209.0]",2018.08.12.14.42.09


In [5]:
ev_row.iloc[0,0]

2018-08-12T14:42:09.000000Z

In [6]:
# Filter
freqmin = 0.02
freqmax = 0.05
corners = 3

# Desired sampling interval
dt = 1.0

# Reduction velocity in km/sec, 0 sets the reference time to origin time
vred = 0

# time before and after reference time, data will be cut before and after the reference time
time_before = 30
time_after = 200

In [7]:
df_catalog.head(10)

,date_time,lat,lon,depth,mag,corr,date_time_id
0,2018-08-12T14:42:09.000000Z,17.11,-100.84,44.3,5.2,"[0.9316, 51.61, 20180812144209.0]",2018.08.12.14.42.09


In [8]:
if vred:
    p = 1/vred
else:
    p = 0
    
st.filter("bandpass",freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
st.taper(max_percentage=0.05)
origin_time = df_catalog.loc[df_catalog['date_time_id'] == evid]['date_time'].values[0]
print(origin_time)

# Trim and decimate the data
for tr in st:
    tr.decimate(factor=int(tr.stats.sampling_rate*dt), strict_length=False, no_filter=True)
    tr.resample(1/dt, strict_length=False, no_filter=True)
    # Recortar aun mas los datos en base a la distancia entre la estacion y el evento
    tr.stats.sac.t1 = origin_time + p*(tr.stats.sac.dist)
    tr.trim(tr.stats.sac.t1-time_before,tr.stats.sac.t1+time_after,pad=True,fill_value=0)
    tr.data = 100*tr.data # m/s to cm/s
    tr.stats.sac.b = -1*(origin_time - tr.stats.starttime)
    tr.stats.sac.o = 0
    # Save final trace using tdmtpy file name format
    sacout = "%s/%s.%s.dat"%(outdir,tr.stats.sac.kstnm,tr.stats.sac.kcmpnm[-1])
    tr.write(sacout,format="SAC")

2018-08-12T14:42:09.000000Z


In [9]:
model = "gil7"
depths = sorted([10,20,round(df_catalog.loc[df_catalog['date_time_id'] == evid]["depth"].values[0])]) # Rango de profundidades para las funciones de Green
npts = int(128) # numero de puntos
t0 = int(0)
ev_st_sub = ev_st_df.loc[ev_st_df['date_time_id'] == evid].copy()
#print(ev_st_sub)

# Directorio de funciones de Green
green_dir = "%s/%s"%(evdir,model)
Path(green_dir).mkdir(parents=True,exist_ok=True)
    
for depth in depths:
    dfile = ("{dist:.0f} {dt:.2f} {npts:d} {t0:d} {vred:.1f}\n")
    dfile_out = "%s/dfile"%evdir
    with open(dfile_out,"w") as f:
        # Distancia con cada estacion
        for i, row in ev_st_sub.iterrows():
            f.write(dfile.format(dist=row.distance,dt=dt,npts=npts,t0=t0,vred=vred))

    # Generate the synthetics
    os.system("hprep96 -M %s.d -d %s -HS %.4f -HR 0 -EQEX"%(model,dfile_out,depth))
    !hspec96
    !hpulse96 -D -i > file96
    !f96tosac -B file96

    greens = ("ZDD","RDD","ZDS","RDS","TDS","ZSS","RSS","TSS","ZEX","REX")

    for i,row in stations_meta.iterrows():      
        for j,grn in enumerate(greens):
            sacin = "B%03d%02d%s.sac"%(i+1,j+1,grn)
            sacout = "%s/%s.%.4f"%(green_dir,row.kstnm,depth)
            tmp = read(sacin,format="SAC")
            tmp.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
            tmp.write("%s.%s"%(sacout,grn),format="SAC") # overwrite

!\rm B*.sac

  0.1953125E-01  0.1000000E+01
        128         1        65
 ieqex=            0  EARTHQUAKE + EXPLOSION
   10
  TOP  OF MODEL IS FREE SURFACE  
  BASE OF MODEL IS HALFSPACE WITH PROPERTIES OF BOTTOM LAYER
 gil7.d
 LAYER             H      P-VEL     S-VEL   DENSITY  
     1          1.000     3.200     1.500     2.280
     2          2.000     4.500     1.400     2.280
     3          1.000     4.800     2.780     2.580
     4          1.000     5.510     3.180     2.580
     5         12.000     6.210     3.400     2.680
     6          8.000     6.890     3.980     3.000
     7          0.000     7.830     4.520     3.260
 Working model
        d         a         b         rho      1/qa      1/qb       bsh    1/qbsh
      1.000     3.200     1.500     2.280  0.001667  0.003333     1.500     2.280
      2.000     4.500     1.400     2.280  0.001667  0.003333     1.400     2.280
      1.000     4.800     2.780     2.580  0.001667  0.003333     2.780     2.580
      1.000     5.510 

In [10]:
headers = dict(datetime=ev_row["date_time"][0],
               longitude=ev_row["lon"][0],
               latitude=ev_row["lat"][0],
               depth=",".join([ "%.4f"%d for d in depths]),
               path_to_data=evdir,
               path_to_green=green_dir,
               green="herrmann",
               components="ZRT", # ZRT y ZNE unicas opciones
               degree=5,
               weight="distance",
               plot=0,
               correlate=0,
              )

pd.options.display.float_format = "{:,.2f}".format
frame = {"station": stations_meta["kstnm"]}
df_out = pd.DataFrame(frame)

df_out["distance"] = ev_st_df["distance"]
df_out["azimuth"] = ev_st_df["azimuth"]
df_out["ts"] = int(30)
df_out["npts"] = npts
df_out["dt"] = dt
df_out["used"] = 1
df_out[["longitude","latitude"]] = stations_meta[["stlo","stla"]]

In [11]:
with open("mtinv.in","w") as f:
    for key, value in headers.items():
        f.write("{0:<15}{1}\n".format(key,value))
    f.write(df_out.to_string(index=False))
    
!cat mtinv.in

datetime       2018-08-12T14:42:09.000000Z
longitude      -100.84
latitude       17.11
depth          10.0000,20.0000,44.0000
path_to_data   Datos_Trim/2018.08.12.14.42.09
path_to_green  Datos_Trim/2018.08.12.14.42.09/gil7
green          herrmann
components     ZRT
degree         5
weight         distance
plot           0
correlate      0
station  distance  azimuth  ts  npts   dt  used  longitude  latitude
   CRIG    169.24   103.92  30   128 1.00     1     -99.30     16.74
   CRIG    169.24   103.92  30   128 1.00     1     -99.30     16.74
   CRIG    169.24   103.92  30   128 1.00     1     -99.30     16.74
   DAIG    126.98    94.26  30   128 1.00     1     -99.65     17.02
   DAIG    126.98    94.26  30   128 1.00     1     -99.65     17.02
   DAIG    126.98    94.26  30   128 1.00     1     -99.65     17.02
   MEIG    157.88    54.98  30   128 1.00     1     -99.62     17.92
   MEIG    157.88    54.98  30   128 1.00     1     -99.62     17.92
   MEIG    157.88    54.98  30   128 1